<p><font size="6" color='grey'> <b>
Machine Learning
</b></font> </br></p>
<p><font size="5" color='grey'> <b>
Ensemble - Stacking 3+1 - Titanic
</b></font> </br></p>

---


# 0  | Install & Import
***

In [ ]:
# Install

In [ ]:
# Import
from pandas import read_excel, DataFrame
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    ConfusionMatrixDisplay,
    classification_report,
)

In [ ]:
# Warnung ausstellen
import warnings
warnings.filterwarnings("ignore")

# 1  | Understand
***


<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Aufgabe verstehen</br>
✅ Daten sammeln</br>
✅ Statistische Analyse (Min, Max, Mean, Korrelation, ...)</br>
✅ Datenvisualisierung (Streudiagramm, Box-Plot, ...)</br>
✅ Prepare Schritte festlegen</br>

<p><font color='black' size="5">
Anwendungsfall
</font></p>

Dies ist der legendäre Titanic ML-Wettbewerb – die beste erste Herausforderung, um in ML-Modellierung einzutauchen.

Die Aufgabe ist einfach: Verwenden Sie maschinelles Lernen, um ein Modell zu erstellen, das vorhersagt, welche Passagiere den Schiffbruch der Titanic überlebt haben.

[Titanic Org](https://www.encyclopedia-titanica.org/)

[DataSet](https://www.openml.org/search?type=data&status=active&id=40945)

[Info](https://www.kaggle.com/competitions/titanic/data)



In [ ]:
df = read_excel(
    "https://raw.githubusercontent.com/ralf-42/ML_Intro/main/02_daten/05_tabellen/titanic.xlsx",
    usecols=["pclass", "survived", "sex", "age", "sibsp", "parch"],
)

In [ ]:
data = df.copy()
target = data.pop("survived")

<p><font color='black' size="5">
EDA (Exploratory Data Analysis) mit Pandas
</font></p>

In [ ]:
data.info()

In [ ]:
data.describe().T

In [ ]:
data.groupby("pclass").count()

In [ ]:
target.value_counts()

In [ ]:
# _ = data.hist(figsize=(15,15))

#  2 | Prepare

<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Train-Test-Split durchführen</br>
✅ Nicht benötigte Features löschen</br>
✅ Datentyp ermitteln/ändern</br>
✅ Missing Values behandeln</br>
✅ Ausreißer behandeln</br>
✅ Kategorischer Features Kodieren</br>
✅ Numerischer Features skalieren</br>
✅ Feature-Engineering (neue Features schaffen)</br>
✅ Dimensionalität reduzieren</br>
✅ Resampling (Over-/Undersampling)</br>
✅ Pipeline erstellen/konfigurieren</br>


<font color='black' size="5">
Datentyp ermitteln
</font>

In [ ]:
all_col = data.columns
num_col = data.select_dtypes(include="number").columns
cat_col = data.select_dtypes(exclude="number").columns

<font color='black' size="5">
Train-Test-Set
</font>

In [ ]:
data_train, data_test, target_train, target_test = train_test_split(
    data, target, test_size=0.20, stratify=target, random_state=42)

<font color='black' size="5">
Missing Values - kategoriale Features
</font>

In [ ]:
imputer_cat = SimpleImputer(strategy="most_frequent")
imputer_cat.fit(data_train[cat_col])
data_train[cat_col] = imputer_cat.transform(data_train[cat_col])
data_test[cat_col] = imputer_cat.transform(data_test[cat_col])

<font color='black' size="5">
Kodierung
</font>

In [ ]:
encoder = OrdinalEncoder()
encoder.fit(data_train[cat_col])
data_train[cat_col] = encoder.transform(data_train[cat_col])
data_test[cat_col] = encoder.transform(data_test[cat_col])

<font color='black' size="5">
Missing Values - numerische Features
</font>

In [ ]:
imputer_num = SimpleImputer(strategy="mean")
imputer_num.fit(data_train[num_col])
data_train[num_col] = imputer_num.transform(data_train[num_col])
data_test[num_col] = imputer_num.transform(data_test[num_col])

<font color='black' size="5">
Skalierung
</font>

In [ ]:
scaler = StandardScaler()
scaler.fit(data_train[all_col])
data_train[all_col] = scaler.transform(data_train[all_col])
data_test[all_col] = scaler.transform(data_test[all_col])

# 3 | Modeling
---

<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Modellauswahl treffen</br>
✅ Pipeline erweitern/konfigurieren</br>
✅ Training durchführen</br>
✅ Hyperparameter Tuning</br>
✅ Cross-Valdiation</br>
✅ Bootstrapping</br>
✅ Regularization</br>

<p><font color='black' size="5">
Aufbau Ensemble
</font></p>

In [ ]:
model_list = [
    ("rf", RandomForestClassifier(n_estimators=10, random_state=42)),
    ("knn", KNeighborsClassifier()),
    ("svc", LinearSVC(random_state=42)),
]

meta_learner = LogisticRegression()

In [ ]:
model = StackingClassifier(estimators=model_list, final_estimator=meta_learner)

In [ ]:
model.fit(data_train, target_train)

# 4 | Evaluate
---

<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Prognose (Train, Test) erstellen</br>
✅ Modellgüte prüfen</br>
✅ Residuenanalyse erstellen</br>
✅ Feature Importance/Selektion prüfen</br>
✅ Robustheitstest erstellen</br>
✅ Modellinterpretation erstellen</br>
✅ Sensitivitätsanalyse erstellen</br>
✅ Kommunikation (Key Takeaways)</br>

<p><font color='black' size="5">
Prognose
</font></p>

In [ ]:
target_train_pred = model.predict(data_train)
target_test_pred = model.predict(data_test)


<p><font color='black' size="5">
Accuracy
</font></p>

In [ ]:
acc_train = accuracy_score(target_train, target_train_pred) * 100
print(f"-- Train -- Accuracy: {acc_train:5.2f}")

In [ ]:
acc_test = accuracy_score(target_test, target_test_pred) * 100
print(f"-- Test -- Accuracy: {acc_test:5.2f}")


<p><font color='black' size="5">
Confusion Matrix
</font></p>

In [ ]:
conf_matrix = confusion_matrix(target_test, target_test_pred)
disp = ConfusionMatrixDisplay(conf_matrix, display_labels=["Not Survived", "Survived"])
disp.plot(cmap="Blues")

In [ ]:
print(
    classification_report(
        target_test, target_test_pred, target_names=["Not Survived", "Survived"]
    )
)

# 5 | Deploy
---

<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Modellexport und -speicherung</br>
✅ Abhängigkeiten und Umgebung</br>
✅ Sicherheit und Datenschutz</br>
✅ In die Produktion integrieren</br>
✅ Tests und Validierung</br>
✅ Dokumentation & Wartung</br>